In [2]:
!pip install demucs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.3 MB/s eta 0:00:00
  Created wheel for demucs: filename=demucs-4.0.1-py3-none-any.whl size=78388 sha256=54ea2c2113ce6b5708b93fd43c58b8da199b21d0fca0a738bea2bb73d4de28ad
  Stored in directory: /root/.cache/pip/wheels/1b/0c/20/a3b3daa1f9b65c8b0445729f94740ec335d0f86f1066c5c414
  Created wheel for julius: filename=julius-0.2.7

In [3]:
from pathlib import Path
import torch
import librosa
import soundfile as sf
from demucs.pretrained import get_model
from demucs.apply import apply_model

In [7]:
# ---------- PATHS ----------
INPUT_PATH = Path("/content/data/input/good for the ghost - Alge.mp3")
OUTPUT_DIR = Path("/content/data/output_demucs_2")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"👉 Fichier d'entrée : {INPUT_PATH.resolve()}")
print(f"👉 Dossier de sortie : {OUTPUT_DIR.resolve()}")

# ---------- CHARGEMENT DU MODÈLE ----------
print("⏳ Chargement du modèle Demucs (htdemucs)...")
model = get_model("htdemucs")

# ---------- LECTURE DE L'AUDIO (librosa AU LIEU de torchaudio) ----------
print("⏳ Chargement de l'audio avec librosa...")
waveform, sr = librosa.load(str(INPUT_PATH), sr=None, mono=False)

# librosa retourne (time) ou (channels, time)
# → on force le format [channels, time]
import numpy as np
if waveform.ndim == 1:
    waveform = np.expand_dims(waveform, axis=0)

# Convertir en tenseur torch : [1, channels, time]
waveform = torch.tensor(waveform).float().unsqueeze(0)

# ---------- SÉPARATION ----------
print("🎧 Séparation en stems...")
sources = apply_model(model, waveform)[0] # [stems, channels, time]

# ---------- SAUVEGARDE avec soundfile ----------
for source_tensor, name in zip(sources, model.sources):
    out_path = OUTPUT_DIR / f"{INPUT_PATH.stem}_{name}.wav"
    sf.write(str(out_path),
             source_tensor.cpu().numpy().T,  # transpose → (time, channels)
             sr)
    print(f"✅ Stem '{name}' sauvegardé : {out_path.resolve()}")

👉 Fichier d'entrée : /content/data/input/good for the ghost - Alge.mp3
👉 Dossier de sortie : /content/data/output_demucs_2
⏳ Chargement du modèle Demucs (htdemucs)...
⏳ Chargement de l'audio avec librosa...
🎧 Séparation en stems...
✅ Stem 'drums' sauvegardé : /content/data/output_demucs_2/good for the ghost - Alge_drums.wav
✅ Stem 'bass' sauvegardé : /content/data/output_demucs_2/good for the ghost - Alge_bass.wav
✅ Stem 'other' sauvegardé : /content/data/output_demucs_2/good for the ghost - Alge_other.wav
✅ Stem 'vocals' sauvegardé : /content/data/output_demucs_2/good for the ghost - Alge_vocals.wav


In [8]:
!pip install musdb museval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.2/963.2 kB 22.2 MB/s eta 0:00:00


In [11]:
import musdb

# Cela va télécharger ~800 Mo de données dans le dossier spécifié
mus = musdb.DB(root="data/musdb18_sample", subsets="test", download=True)

# Pour vérifier que ça fonctionne :
print(f"Nombre de pistes chargées : {len(mus)}")
track = mus[0]
print(f"Titre : {track.name}")

100%|██████████| 140M/140M [00:02<00:00, 73.6MB/s]


Nombre de pistes chargées : 50
Titre : AM Contra - Heart Peripheral


In [12]:
import musdb
import museval
import torch
import numpy as np
from demucs.pretrained import get_model
from demucs.apply import apply_model

# 1. Configuration
# 'subsets="test"' pour évaluer sur le set de test officiel
# 'download=True' si tu ne l'as pas encore sur ton disque
mus = musdb.DB(root="data/musdb18_sample", subsets="test", download=True)

# 2. Chargement du modèle Demucs
device = "cuda" if torch.cuda.is_available() else "cpu"
model = get_model("htdemucs")
model.to(device)
model.eval()

print(f"🚀 Début de l'évaluation sur {len(mus)} morceaux...")

for track in mus:
    print(f"🎧 Évaluation de : {track.name}")

    # Préparation de l'audio (Normalisation pour Demucs)
    # Shape attendue par Demucs : [1, channels, time]
    waveform = torch.from_numpy(track.audio.T).float().unsqueeze(0).to(device)

    with torch.no_grad():
        # Séparation
        sources = apply_model(model, waveform)[0]

    # 3. Préparation des dictionnaires pour museval
    # museval attend des dictionnaires : { 'nom_piste': numpy_array (samples, channels) }
    references = {
        'drums': track.targets['drums'].audio,
        'bass': track.targets['bass'].audio,
        'other': track.targets['other'].audio,
        'vocals': track.targets['vocals'].audio,
    }

    estimates = {}
    for i, name in enumerate(model.sources):
        # On repasse en (samples, channels) et cpu
        estimates[name] = sources[i].cpu().numpy().T

    # 4. Calcul des métriques
    # 'evaluate' compare les dictionnaires et calcule SDR, ISR, SIR, SAR
    scores = museval.eval_mus_track(track, estimates, output_dir="results")

    print(f"✅ Terminé pour {track.name}")
    print(scores)

# 5. Synthèse finale
# museval crée des fichiers .json dans le dossier 'results'
# Tu peux ensuite utiliser la commande : 'python -m museval.aggregate results/'

🚀 Début de l'évaluation sur 50 morceaux...
🎧 Évaluation de : AM Contra - Heart Peripheral
✅ Terminé pour AM Contra - Heart Peripheral
vocals          ==> SDR:  11.298  SIR:  15.503  ISR:  16.312  SAR:  13.005  
drums           ==> SDR:   7.867  SIR:  11.244  ISR:  13.380  SAR:   7.660  
bass            ==> SDR:   3.804  SIR:   3.535  ISR:  10.766  SAR:   7.365  
other           ==> SDR:   0.623  SIR:   1.478  ISR:   0.974  SAR:   0.488  

🎧 Évaluation de : Al James - Schoolboy Facination
✅ Terminé pour Al James - Schoolboy Facination
vocals          ==> SDR:   9.319  SIR:  14.115  ISR:  17.216  SAR:  10.800  
drums           ==> SDR:   6.414  SIR:   7.692  ISR:  10.659  SAR:   6.169  
bass            ==> SDR:  14.713  SIR:  22.947  ISR:  17.833  SAR:  16.301  
other           ==> SDR:   5.744  SIR:   9.851  ISR:   8.887  SAR:   5.746  

🎧 Évaluation de : Angels In Amplifiers - I'm Alright
✅ Terminé pour Angels In Amplifiers - I'm Alright
vocals          ==> SDR:  10.497  SIR:  16.926  

In [16]:
import numpy as np
import soundfile as sf
from pathlib import Path

def generate_test_stems(output_dir="data/test_synth"):
    path = Path(output_dir)
    path.mkdir(parents=True, exist_ok=True)

    sr = 44100
    duration = 100.0  # secondes
    t = np.linspace(0, duration, int(sr * duration), endpoint=False)

    # 1. DRUMS : Des pulses toutes les 0.5s (Kick synthétique)
    drums = np.zeros_like(t)
    for i in range(0, len(t), int(sr * 0.5)):
        # Un petit burst de sinus décroissant pour simuler un impact
        end = min(i + int(sr * 0.1), len(t))
        drums[i:end] = np.sin(2 * np.pi * 60 * t[:end-i]) * np.exp(-10 * t[:end-i])

    # 2. BASS : Une onde sinusoïdale pure à 50Hz
    bass = 0.5 * np.sin(2 * np.pi * 50 * t)

    # 3. VOCALS : Une onde en dents de scie (timbre plus riche) à 440Hz
    from scipy import signal
    vocals = 0.3 * signal.sawtooth(2 * np.pi * 440 * t)

    # Sauvegarde des Stems (mono vers stéréo pour Demucs)
    stems = {'drums': drums, 'bass': bass, 'vocals': vocals}
    for name, data in stems.items():
        # Conversion en stéréo [time, channels]
        stereo = np.stack([data, data], axis=1)
        sf.write(path / f"{name}.wav", stereo, sr)

    # 4. MIX : La somme des trois
    mix = drums + bass + vocals
    # Normalisation pour éviter la saturation (clipping)
    mix = mix / np.max(np.abs(mix))
    sf.write(path / "mix_test.wav", np.stack([mix, mix], axis=1), sr)

    print(f"✅ Fichiers générés dans {path.resolve()}")

generate_test_stems()

✅ Fichiers générés dans /content/data/test_synth


In [18]:
# ---------- PATHS ----------
INPUT_PATH = Path("/content/data/test_synth/mix_test.wav")
OUTPUT_DIR = Path("/content/data/output_demucs")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"👉 Fichier d'entrée : {INPUT_PATH.resolve()}")
print(f"👉 Dossier de sortie : {OUTPUT_DIR.resolve()}")

# ---------- CHARGEMENT DU MODÈLE ----------
print("⏳ Chargement du modèle Demucs (htdemucs)...")
model = get_model("htdemucs")

# ---------- LECTURE DE L'AUDIO (librosa AU LIEU de torchaudio) ----------
print("⏳ Chargement de l'audio avec librosa...")
waveform, sr = librosa.load(str(INPUT_PATH), sr=None, mono=False)

# librosa retourne (time) ou (channels, time)
# → on force le format [channels, time]
import numpy as np
if waveform.ndim == 1:
    waveform = np.expand_dims(waveform, axis=0)

# Convertir en tenseur torch : [1, channels, time]
waveform = torch.tensor(waveform).float().unsqueeze(0)

# ---------- SÉPARATION ----------
print("🎧 Séparation en stems...")
sources = apply_model(model, waveform)[0] # [stems, channels, time]

# ---------- SAUVEGARDE avec soundfile ----------
for source_tensor, name in zip(sources, model.sources):
    out_path = OUTPUT_DIR / f"{INPUT_PATH.stem}_{name}.wav"
    sf.write(str(out_path),
             source_tensor.cpu().numpy().T,  # transpose → (time, channels)
             sr)
    print(f"✅ Stem '{name}' sauvegardé : {out_path.resolve()}")

👉 Fichier d'entrée : /content/data/test_synth/mix_test.wav
👉 Dossier de sortie : /content/data/output_demucs
⏳ Chargement du modèle Demucs (htdemucs)...
⏳ Chargement de l'audio avec librosa...
🎧 Séparation en stems...
✅ Stem 'drums' sauvegardé : /content/data/output_demucs/mix_test_drums.wav
✅ Stem 'bass' sauvegardé : /content/data/output_demucs/mix_test_bass.wav
✅ Stem 'other' sauvegardé : /content/data/output_demucs/mix_test_other.wav
✅ Stem 'vocals' sauvegardé : /content/data/output_demucs/mix_test_vocals.wav


In [32]:
# Exemple de comparaison rapide pour la Basse
import museval

# Charger l'originale (ref) et la sortie de Demucs (est)
# Ils doivent avoir exactement la même taille (shape)
ref_bass = sf.read("data/test_synth/bass.wav")[0][:, :, np.newaxis]
est_bass = sf.read("data/output_demucs/mix_test_bass.wav")[0][:, :, np.newaxis]

# On transpose(dimension_source, dimension_samples, dimension_channels) pour avoir la même dimension que BSSeval
ref_bass_corrected = ref_bass.transpose(2, 0, 1)
est_bass_corrected = est_bass.transpose(2, 0, 1)

sdr, isr, sir, sar = museval.evaluate(ref_bass_corrected, est_bass_corrected)
print(f"SDR Synthétique Basse : {np.nanmedian(sdr):.2f} dB")

ref_drums = sf.read("data/test_synth/drums.wav")[0][:, :, np.newaxis]
est_drums = sf.read("data/output_demucs/mix_test_drums.wav")[0][:, :, np.newaxis]

# On transpose(dimension_source, dimension_samples, dimension_channels) pour avoir la même dimension que BSSeval
ref_drums_corrected = ref_drums.transpose(2, 0, 1)
est_drums_corrected = est_drums.transpose(2, 0, 1)

sdr, isr, sir, sar = museval.evaluate(ref_drums_corrected, est_drums_corrected)
print(f"SDR Synthétique Drums : {np.nanmedian(sdr):.2f} dB")

ref_vocals = sf.read("data/test_synth/vocals.wav")[0][:, :, np.newaxis]
est_vocals = sf.read("data/output_demucs/mix_test_vocals.wav")[0][:, :, np.newaxis]

# On transpose(dimension_source, dimension_samples, dimension_channels) pour avoir la même dimension que BSSeval
ref_vocals_corrected = ref_vocals.transpose(2, 0, 1)
est_vocals_corrected = est_vocals.transpose(2, 0, 1)

sdr, isr, sir, sar = museval.evaluate(ref_vocals_corrected, est_vocals_corrected)
print(f"SDR Synthétique Vocals : {np.nanmedian(sdr):.2f} dB")

SDR Synthétique Basse : 7.33 dB
SDR Synthétique Drums : 6.92 dB
SDR Synthétique Vocals : 0.01 dB
